In [ ]:
import googlemaps
from datetime import datetime
import pandas as pd
from IPython.core.display import clear_output
from random import randint
from time import time 
import requests
from requests import get
from warnings import warn 
from bs4 import BeautifulSoup
from IPython.core.display import clear_output
from time import sleep
from itertools import islice

In [ ]:
# assign filepath to csv variable
csv = "geocode_craigslist_rentals_2.csv"
# read csv 
df = pd.read_csv(csv)
# set index
index_list = list(range(len(df.index)))
df['index'] = index_list
df = df.set_index(['index'])
# use islice function to select only within a range of 2500 rows to iterate thorugh
# as this is the limit for google api
for index, row in islice(df.iterrows(),5000, 7500):
    # only make calls to Google Geocodie API for rows that have no geocode coordinates and DO HAVE (False indicates address present)
    # an address
    if (row['geocode_lat'] == "none") & (row['geocode_lon'] == "none") & (row['address_check'] == False):
    # set count variable to track status of requests that script has made 
    count = 0 
        try:
            # sleep variable to wait 2 seconds between each address so as not to slow things down for Google
            sleep((2))
            # read in Google Geocode Key which can be obtained for free from Google
            google_key = " "
            gmaps = googlemaps.Client(key= google_key)
            # make address a string to be read by google geocode
            address = str(row['address']
            # set result of geocode in a variable
            geocode_result = gmaps.geocode(address)
            # get latitude from geocode result
            latitude = geocode_result[0]['geometry']['location']['lat']
            # get longitude from geocode result
            longitude = geocode_result[0]['geometry']['location']['lng'] 
            # set longitude at appropriate row where address was geocoded
            df.loc[index, "geocode_lon"] = str(longitude)
            # set latitude at appropriate row where address was geocoded
            df.loc[index, "geocode_lat"] = str(latitude)
            # increment count variable
            count += 1
            # print status of script
            print ('Request: {}'.format(count))
            clear_output(wait = True)
        except:
            # indicate error if geocode request doesn't work
            df.loc[index, "geocode_lon"] = "error"
            df.loc[index, "geocode_lat"] = "error"
            # increment and display request count
            count += 1
            print ('Request: {}'.format(count))
            clear_output(wait = True)
# save data in same csv, so as to call on this csv to do 
# further geocode 2500 rows at a time
df.to_csv('geocode_craigslist_rentals_2.csv', header=True)